In [2]:
import pandas as pd
from __future__ import division
import csv
from datetime import datetime
from collections import Counter

In [3]:
import matplotlib.pyplot as plt

import stuff

In [4]:
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt')

In [27]:
list_data =[]
list_weeks = ['160102', '160109', '160116','160130','160206','160213',
              '160220','160227','160305','160312','160319','160326','160402']
for u in list_weeks:
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_' + u + '.txt'
    print(url)
    list_data.append(pd.read_csv(url))

data = pd.concat(list_data)

http://web.mta.info/developers/data/nyct/turnstile/turnstile_160102.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160109.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160116.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160130.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160206.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160213.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160220.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160227.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160305.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160312.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160319.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160326.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160402.txt


read data in + merge data

In [28]:
data.columns = data.columns.str.replace(' ','')

remove spaces from exits column

In [29]:
data["DATETIME"] = data["DATE"] + ' ' + data["TIME"]

In [30]:
data['DATETIME'] = pd.to_datetime(data.DATETIME, format='%m/%d/%Y %H:%M:%S')

create datetime and set format

In [31]:
data['HOUR'] = data.DATETIME.apply(lambda x: x.hour)

In [32]:
data['WEEKDAY']=data.DATETIME.apply(lambda x: 'Weekday' if 0<=x.weekday()<5 else 'Weekend')

#same as above in 2 steps
#weekdaycheck = lambda x: 'Weekday' if 0<=x.weekday()<5 else 'Weekend'
#data['WEEKDAY']=data.DATETIME.apply(weekdaycheck)


In [33]:
daysofweek = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

whatdayofweek = lambda x: daysofweek[x.weekday()]


data['DAYOFWEEK'] = data.DATETIME.apply(whatdayofweek)
#data['DAYOFWEEKVALUE'] = data.DATETIME.apply(dayofweekvalue)

create day of week by applying function to DATETIME

In [70]:
data['dENTRIES'] = data.ENTRIES.shift(-1) - data.ENTRIES
data['dEXITS'] = data.EXITS.shift(-1) - data.EXITS

###for considering deltas as time(i+1)-time(i)

create deltas by shifting from one above

In [12]:
#data['dENTRIES'] = data.ENTRIES - data.ENTRIES.shift(1)
#data['dEXITS'] = data.EXITS - data.EXITS.shift(1)
#data['dTOTAL'] = data.dENTRIES+ data.dEXITS

###this is for considering deltas as time(i)-time(i-1)

In [52]:
data.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS', 'DATETIME', 'HOUR', 'WEEKDAY', 'DAYOFWEEK',
       'dENTRIES', 'dEXITS', 'dTOTAL'],
      dtype='object')

In [71]:
data.dENTRIES.fillna(0, inplace = True)

In [72]:
data.dEXITS.fillna(0, inplace = True)

In [73]:
data['dTOTAL'] = data.dENTRIES+ data.dEXITS


In [75]:
data[data.dTOTAL.isnull()].shape

(0, 18)

In [76]:
datatest = data[(data.HOUR ==0)|(data.HOUR ==1)|(data.HOUR==2)|(data.HOUR==3)]
# same as mask

In [77]:
datatest.shape

(411197, 18)

In [44]:
datatest.shape

(220559, 18)

filter based on time we care about

In [78]:
datatest2 = datatest[(datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)]
# mask2 = (datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)


In [79]:
datatest2.shape

(407288, 18)

filter a second time based on change in deltas to account for switching machine/station etc.  
Lose out on 411197 - 407288 = 3909 total entries  
3909 is 0.95% of our data that we lose

In [80]:
final_weekday_or_end = datatest2.groupby([datatest2.WEEKDAY, datatest2.STATION]).sum()

In [81]:
weekend_or_day_final =  (final_weekday_or_end
    .reset_index()
    .sort_values(["WEEKDAY", "dTOTAL"], ascending=[True, False])
)

In [82]:
weekend_or_day_final = weekend_or_day_final.drop(['ENTRIES', 'EXITS', 'HOUR', 'dENTRIES', 'dEXITS'],1)

In [83]:
for r in range(weekend_or_day_final.shape[0]):
    if weekend_or_day_final.ix[r,'WEEKDAY'] =='Weekday':
        weekend_or_day_final.ix[r,'dTOTALACTUAL'] = weekend_or_day_final.ix[r, 'dTOTAL']/5
    else:
        weekend_or_day_final.ix[r,'dTOTALACTUAL'] = weekend_or_day_final.ix[r, 'dTOTAL']/2

In [84]:
weekend_or_day_final = weekend_or_day_final.drop(['dTOTAL'],1)


In [85]:
weekend_or_day_final.head()

,WEEKDAY,STATION,dTOTALACTUAL
61,Weekday,34 ST-PENN STA,120780.8
68,Weekday,42 ST-PORT AUTH,95942.2
248,Weekday,JAMAICA CENTER,80605.8
213,Weekday,FLUSHING-MAIN,79617.6
229,Weekday,GRD CNTRL-42 ST,73518.2


weekend_or_day_final is the FINAL dataframe for weekday vs weekend

In [86]:
finaldayofweek = datatest2.groupby([datatest2.DAYOFWEEK,datatest2.STATION]).sum()

In [87]:
day_of_week = finaldayofweek.reset_index()

In [88]:
whatdayindex = lambda x: daysofweek.index(x)

In [89]:
day_of_week['DAYOFWEEKVALUE'] = day_of_week.DAYOFWEEK.apply(whatdayindex)  


day_of_week_final = day_of_week.sort_values(['DAYOFWEEKVALUE','dTOTAL'], ascending = [True, False])
#data['HOUR'] = data.DATETIME.apply(lambda x: x.hour)

In [90]:
day_of_week_final = day_of_week_final.drop(['ENTRIES', 'EXITS', 'HOUR', 'dENTRIES', 'dEXITS', 'DAYOFWEEKVALUE'],1)



#df = df.drop('column_name', 1)

In [92]:
day_of_week_final.dTOTAL = day_of_week_final.dTOTAL/13


In [97]:
testsunday = day_of_week_final[(day_of_week_final.DAYOFWEEK == 'Sunday')]

In [109]:
testmonday = day_of_week_final[(day_of_week_final.DAYOFWEEK == 'Monday')]

testsaturday = day_of_week_final[(day_of_week_final.DAYOFWEEK == 'Saturday')]

testtuesday = day_of_week_final[(day_of_week_final.DAYOFWEEK == 'Tuesday')]

testwed = day_of_week_final[(day_of_week_final.DAYOFWEEK == 'Wednesday')]

day_of_week_final is FINAL dataframe for each day of week

In [112]:
testwed.head()

,DAYOFWEEK,STATION,dTOTAL
2299,Wednesday,34 ST-PENN STA,9550.153846
2306,Wednesday,42 ST-PORT AUTH,7322.923077
2297,Wednesday,34 ST-HERALD SQ,6769.076923
2486,Wednesday,JAMAICA CENTER,6564.000000
2451,Wednesday,FLUSHING-MAIN,6313.384615


In [110]:
testtuesday.head()

,DAYOFWEEK,STATION,dTOTAL
1926,Tuesday,34 ST-PENN STA,9406.384615
1933,Tuesday,42 ST-PORT AUTH,7352.846154
2113,Tuesday,JAMAICA CENTER,6499.769231
2078,Tuesday,FLUSHING-MAIN,6309.692308
1924,Tuesday,34 ST-HERALD SQ,5574.538462


In [104]:
testsunday.head()

,DAYOFWEEK,STATION,dTOTAL
1133,Sunday,14 ST-UNION SQ,5949.615385
1477,Sunday,W 4 ST-WASH SQ,4322.461538
1187,Sunday,42 ST-PORT AUTH,4176.384615
1180,Sunday,34 ST-PENN STA,3282.000000
1128,Sunday,125 ST,3190.846154


In [106]:
testmonday.head()

,DAYOFWEEK,STATION,dTOTAL
434,Monday,34 ST-PENN STA,8377.538462
441,Monday,42 ST-PORT AUTH,6874.000000
621,Monday,JAMAICA CENTER,5968.076923
586,Monday,FLUSHING-MAIN,5920.384615
482,Monday,86 ST,4986.230769


In [108]:
testsaturday.head()

,DAYOFWEEK,STATION,dTOTAL
760,Saturday,14 ST-UNION SQ,5888.076923
814,Saturday,42 ST-PORT AUTH,4588.846154
807,Saturday,34 ST-PENN STA,4433.615385
1104,Saturday,W 4 ST-WASH SQ,4339.846154
1094,Saturday,TIMES SQ-42 ST,3738.000000
